#Passo a passo pra coletar os dados:

1. Entra no projeto APIV2 do Pycharm e baixa os dados no período de 1 ano;
2. Junta os datasets de clientes e pedidos;
3. Manda o dataset desses clientes pro Gabriel para pegar os CNPJs da base pública com esses clientes (pra terem as mesmas colunas)(essa etapa no código seria: verificar e retornar os CNPJs dos clientes secundários da base pública);
4. Pegar dos dados públicos o estado que mais tem clientes secundários para colocar os CNAES secundários (base CNAEs);
5. Colocar target nos clientes secundários como 1 e outros clientes que ainda não são clientes secundários como 0;
6. Fazer preprocessamento dos dados.

Depois dessas etapas o dataset estará pronto pra fazer o kmeans
(O notebook atualizado está com o Gabriel)

In [1]:
import pandas as pd
import numpy as np

## Etapa 2:

In [2]:
# Nesse caso foi escolhido baixar de 2 em 2 meses lá na etapa 1
# por isso tem 6 datasets de cada

df_cli1 = pd.read_csv('/content/drive/MyDrive/Projetos_ion/ION_Prospeccao_Inteligente/datasets/Recife_Doces/APIV2_recifedoces/df_cli_recife1.csv')
df_cli2 = pd.read_csv('/content/drive/MyDrive/Projetos_ion/ION_Prospeccao_Inteligente/datasets/Recife_Doces/APIV2_recifedoces/df_cli_recife2.csv')
df_cli3 = pd.read_csv('/content/drive/MyDrive/Projetos_ion/ION_Prospeccao_Inteligente/datasets/Recife_Doces/APIV2_recifedoces/df_cli_recife3.csv')
df_cli4 = pd.read_csv('/content/drive/MyDrive/Projetos_ion/ION_Prospeccao_Inteligente/datasets/Recife_Doces/APIV2_recifedoces/df_cli_recife4.csv')
df_cli5 = pd.read_csv('/content/drive/MyDrive/Projetos_ion/ION_Prospeccao_Inteligente/datasets/Recife_Doces/APIV2_recifedoces/df_cli_recife.csv')
df_cli6 = pd.read_csv('/content/drive/MyDrive/Projetos_ion/ION_Prospeccao_Inteligente/datasets/Recife_Doces/APIV2_recifedoces/df_cli_recife6.csv')

df_order1 = pd.read_csv('/content/drive/MyDrive/Projetos_ion/ION_Prospeccao_Inteligente/datasets/Recife_Doces/APIV2_recifedoces/df_order_recife1.csv')
df_order2 = pd.read_csv('/content/drive/MyDrive/Projetos_ion/ION_Prospeccao_Inteligente/datasets/Recife_Doces/APIV2_recifedoces/df_order_recife2.csv')
df_order3 = pd.read_csv('/content/drive/MyDrive/Projetos_ion/ION_Prospeccao_Inteligente/datasets/Recife_Doces/APIV2_recifedoces/df_order_recife3.csv')
df_order4 = pd.read_csv('/content/drive/MyDrive/Projetos_ion/ION_Prospeccao_Inteligente/datasets/Recife_Doces/APIV2_recifedoces/df_order_recife4.csv')
df_order5 = pd.read_csv('/content/drive/MyDrive/Projetos_ion/ION_Prospeccao_Inteligente/datasets/Recife_Doces/APIV2_recifedoces/df_order_recife5.csv')
df_order6 = pd.read_csv('/content/drive/MyDrive/Projetos_ion/ION_Prospeccao_Inteligente/datasets/Recife_Doces/APIV2_recifedoces/df_order_recife6.csv')

FileNotFoundError: ignored

In [ ]:
df_cli1.dropna(subset=['CNPJ'], inplace=True)
df_cli2.dropna(subset=['CNPJ'], inplace=True)
df_cli3.dropna(subset=['CNPJ'], inplace=True)
df_cli4.dropna(subset=['CNPJ'], inplace=True)
df_cli5.dropna(subset=['CNPJ'], inplace=True)
df_cli6.dropna(subset=['CNPJ'], inplace=True)

In [ ]:
df_cli = pd.concat([df_cli1, df_cli2,df_cli3, df_cli4, df_cli5, df_cli6])

In [ ]:
df_order = pd.concat([df_order1, df_order2, df_order3, df_order4, df_order5, df_order6])
df_order = df_order.drop_duplicates()
dfo_cli = df_order.groupby('CODCLI').nunique()
dfo_cli.reset_index(inplace=True)

In [ ]:
df_order_cli = pd.merge(df_cli, dfo_cli, how='left', on='CODCLI')

In [ ]:
df_order_cli.head()

Depois disso, salvaria esse último dataset e mandaria pro Gabriel adicionar as colunas dos dados públicos e devolver para fazermos a etapa 4.

## Etapa 4:

### Features:

- Capital social (merge com a tabela de mepresas);
- Porte (merge com a tabela de mepresas);
- Nº de CNAEs secundários (contar quantos secundarios tem na base de CNPJ);
- CNAE separado por: Seção, Divisão, Grupo, Classe e Subclasse;
- CNAEs secundários separados por: Seção, Divisão, Grupo, Classe e Subclasse (limitado a 5 CNAEs);
- Idade da empresa (usar a feature 'data_inicio_atividade').

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Projetos_ion/ION_Prospeccao_Inteligente/datasets/VetBr/teste_uf_vetbr.csv', dtype=str)

In [ ]:
df.head()

,cnpj,cnpj_basico,identificador_matriz_filial,data_inicio_atividade,cnae,cnae_fiscal_principal,cnae_fiscal_secundaria,tipo_logradouro,logradouro,numero,complemento,bairro,cep,uf,municipio,capital_social,porte
0,35.471.883/0001-00,35471883,MATRIZ,20191111,4789004,Comércio varejista de animais vivos e de artig...,"4712100,4713002,4744099,4771704,4789099",AVENIDA,ESAU LACERDA,2103,NaN,MUTAMBA,62810000,CE,ICAPUI,5000.0,1
1,35.562.380/0001-32,35562380,MATRIZ,20191120,4789004,Comércio varejista de animais vivos e de artig...,"4771704,4772500,4781400,9609208",RUA,PRIMEIRO DE SETEMBRO,265,NaN,CENTRO,8715080,SP,MOGI DAS CRUZES,10000.0,1
2,35.589.544/0001-15,35589544,MATRIZ,20191122,7500100,Atividades veterinárias,"4789004,9609207,9609208",RUA,VICENTE LEPORACE,1534,NaN,CAMPO BELO,4619034,SP,SAO PAULO,100000.0,3
3,29.019.542/0001-98,29019542,MATRIZ,20171107,7500100,Atividades veterinárias,"4771704,4789004,8599604,9609207,9609208",RUA,PRESIDENTE GETULIO VARGAS,345,SALA 1,CENTRO,12010500,SP,TAUBATE,5000.0,1
4,35.681.862/0001-01,35681862,MATRIZ,20191202,4789004,Comércio varejista de animais vivos e de artig...,"4723700,4744001,4763604,4789002,4789005",RUA,JOAQUIM ALVES DE OLIVEIRA,57,NaN,VILA DE FATIMA,12260000,SP,PARAIBUNA,10000.0,1


In [ ]:
cnaes = pd.read_csv('/content/drive/MyDrive/Projetos_ion/ION_Prospeccao_Inteligente/datasets/CNPJ_metadados/cnaes_divisao.csv', dtype=str)

In [ ]:
cnaes.head()

,cnae,Denominação,Seção,Divisão,Grupo,Classe,Subclasse,Seção_str,Divisão_str,Grupo_str,Classe_str,Subclasse_str
0,0111301,Cultivo de arroz,A,01,01.1,01.11-3,0111-3/01,Cultivos e semicultivos da aquicultura em água...,Caça e serviços relacionados,Cultivo de outras plantas de lavoura temporári...,Cultivo de outros cereais não especificados an...,Cultivo de arroz
1,0111302,Cultivo de milho,A,01,01.1,01.11-3,0111-3/02,Cultivos e semicultivos da aquicultura em água...,Caça e serviços relacionados,Cultivo de outras plantas de lavoura temporári...,Cultivo de outros cereais não especificados an...,Cultivo de milho
2,0111303,Cultivo de trigo,A,01,01.1,01.11-3,0111-3/03,Cultivos e semicultivos da aquicultura em água...,Caça e serviços relacionados,Cultivo de outras plantas de lavoura temporári...,Cultivo de outros cereais não especificados an...,Cultivo de trigo
3,0111399,Cultivo de outros cereais não especificados an...,A,01,01.1,01.11-3,0111-3/99,Cultivos e semicultivos da aquicultura em água...,Caça e serviços relacionados,Cultivo de outras plantas de lavoura temporári...,Cultivo de outros cereais não especificados an...,Cultivo de outros cereais não especificados an...
4,0112101,Cultivo de algodão herbáceo,A,01,01.1,01.12-1,0112-1/01,Cultivos e semicultivos da aquicultura em água...,Caça e serviços relacionados,Cultivo de outras plantas de lavoura temporári...,Cultivo de outras fibras de lavoura temporária...,Cultivo de algodão herbáceo


In [ ]:
cnaes.shape

(1331, 12)

In [ ]:
df_merge = pd.merge(df, cnaes, how = 'left', on = 'cnae')

In [ ]:
df_merge.head()

,cnpj,cnpj_basico,identificador_matriz_filial,data_inicio_atividade,cnae,cnae_fiscal_principal,cnae_fiscal_secundaria,tipo_logradouro,logradouro,numero,...,Seção,Divisão,Grupo,Classe,Subclasse,Seção_str,Divisão_str,Grupo_str,Classe_str,Subclasse_str
0,35.471.883/0001-00,35471883,MATRIZ,20191111,4789004,Comércio varejista de animais vivos e de artig...,"4712100,4713002,4744099,4771704,4789099",AVENIDA,ESAU LACERDA,2103,...,G,47,47.8,47.89-0,4789-0/04,Comércio varejista de outros produtos não espe...,Comércio varejista de outros produtos não espe...,Comércio varejista de outros produtos não espe...,Comércio varejista de outros produtos não espe...,Comércio varejista de animais vivos e de artig...
1,35.562.380/0001-32,35562380,MATRIZ,20191120,4789004,Comércio varejista de animais vivos e de artig...,"4771704,4772500,4781400,9609208",RUA,PRIMEIRO DE SETEMBRO,265,...,G,47,47.8,47.89-0,4789-0/04,Comércio varejista de outros produtos não espe...,Comércio varejista de outros produtos não espe...,Comércio varejista de outros produtos não espe...,Comércio varejista de outros produtos não espe...,Comércio varejista de animais vivos e de artig...
2,35.589.544/0001-15,35589544,MATRIZ,20191122,7500100,Atividades veterinárias,"4789004,9609207,9609208",RUA,VICENTE LEPORACE,1534,...,M,75,75.0,75.00-1,7500-1/00,Atividades veterinárias,Atividades veterinárias,Atividades veterinárias,Atividades veterinárias,Atividades veterinárias
3,29.019.542/0001-98,29019542,MATRIZ,20171107,7500100,Atividades veterinárias,"4771704,4789004,8599604,9609207,9609208",RUA,PRESIDENTE GETULIO VARGAS,345,...,M,75,75.0,75.00-1,7500-1/00,Atividades veterinárias,Atividades veterinárias,Atividades veterinárias,Atividades veterinárias,Atividades veterinárias
4,35.681.862/0001-01,35681862,MATRIZ,20191202,4789004,Comércio varejista de animais vivos e de artig...,"4723700,4744001,4763604,4789002,4789005",RUA,JOAQUIM ALVES DE OLIVEIRA,57,...,G,47,47.8,47.89-0,4789-0/04,Comércio varejista de outros produtos não espe...,Comércio varejista de outros produtos não espe...,Comércio varejista de outros produtos não espe...,Comércio varejista de outros produtos não espe...,Comércio varejista de animais vivos e de artig...


In [ ]:
df_merge['Grupo'] = df_merge['Grupo'].str.replace('.','')
df_merge['Classe'] = df_merge['Classe'].str.replace('.','')
df_merge['Classe'] = df_merge['Classe'].str.replace('-','')
df_merge['Subclasse'] = df_merge['Subclasse'].str.replace('-','')
df_merge['Subclasse'] = df_merge['Subclasse'].str.replace('/','')

<ipython-input-11-34af28c405e3>:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_merge['Grupo'] = df_merge['Grupo'].str.replace('.','')
<ipython-input-11-34af28c405e3>:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_merge['Classe'] = df_merge['Classe'].str.replace('.','')


In [ ]:
from datetime import datetime

now = datetime.now()
df_merge['idade_empresa'] = df_merge['data_inicio_atividade'].astype(str).apply(lambda x: (now - datetime.strptime(x, '%Y%m%d')).days)

df_merge.head()

,cnpj,cnpj_basico,identificador_matriz_filial,data_inicio_atividade,cnae,cnae_fiscal_principal,cnae_fiscal_secundaria,tipo_logradouro,logradouro,numero,...,Divisão,Grupo,Classe,Subclasse,Seção_str,Divisão_str,Grupo_str,Classe_str,Subclasse_str,idade_empresa
0,35.471.883/0001-00,35471883,MATRIZ,20191111,4789004,Comércio varejista de animais vivos e de artig...,"4712100,4713002,4744099,4771704,4789099",AVENIDA,ESAU LACERDA,2103,...,47,478,47890,4789004,Comércio varejista de outros produtos não espe...,Comércio varejista de outros produtos não espe...,Comércio varejista de outros produtos não espe...,Comércio varejista de outros produtos não espe...,Comércio varejista de animais vivos e de artig...,1219
1,35.562.380/0001-32,35562380,MATRIZ,20191120,4789004,Comércio varejista de animais vivos e de artig...,"4771704,4772500,4781400,9609208",RUA,PRIMEIRO DE SETEMBRO,265,...,47,478,47890,4789004,Comércio varejista de outros produtos não espe...,Comércio varejista de outros produtos não espe...,Comércio varejista de outros produtos não espe...,Comércio varejista de outros produtos não espe...,Comércio varejista de animais vivos e de artig...,1210
2,35.589.544/0001-15,35589544,MATRIZ,20191122,7500100,Atividades veterinárias,"4789004,9609207,9609208",RUA,VICENTE LEPORACE,1534,...,75,750,75001,7500100,Atividades veterinárias,Atividades veterinárias,Atividades veterinárias,Atividades veterinárias,Atividades veterinárias,1208
3,29.019.542/0001-98,29019542,MATRIZ,20171107,7500100,Atividades veterinárias,"4771704,4789004,8599604,9609207,9609208",RUA,PRESIDENTE GETULIO VARGAS,345,...,75,750,75001,7500100,Atividades veterinárias,Atividades veterinárias,Atividades veterinárias,Atividades veterinárias,Atividades veterinárias,1953
4,35.681.862/0001-01,35681862,MATRIZ,20191202,4789004,Comércio varejista de animais vivos e de artig...,"4723700,4744001,4763604,4789002,4789005",RUA,JOAQUIM ALVES DE OLIVEIRA,57,...,47,478,47890,4789004,Comércio varejista de outros produtos não espe...,Comércio varejista de outros produtos não espe...,Comércio varejista de outros produtos não espe...,Comércio varejista de outros produtos não espe...,Comércio varejista de animais vivos e de artig...,1198


In [ ]:
lista_cnae = []
for cnae_sec in df_merge['cnae_fiscal_secundaria']:
  lista_cnae.append(str(cnae_sec).split(','))
lista_cnae

[['4712100', '4713002', '4744099', '4771704', '4789099'],
 ['4771704', '4772500', '4781400', '9609208'],
 ['4789004', '9609207', '9609208'],
 ['4771704', '4789004', '8599604', '9609207', '9609208'],
 ['4723700', '4744001', '4763604', '4789002', '4789005'],
 ['4742300',
  '4744003',
  '4773300',
  '4772500',
  '9609208',
  '4763604',
  '4755502',
  '0322104',
  '0159802',
  '0321304',
  '3299099',
  '4744001',
  '4744099'],
 ['4789004'],
 ['4623109', '4692300', '4744001', '4771704', '4789002'],
 ['4623106', '4683400', '4763604', '4782201', '4789004'],
 ['4789004'],
 ['4789004', '9609208'],
 ['4623109', '4644302', '4744001', '4771704', '4789002', '7500100', '9609208'],
 ['9609208'],
 ['4771704', '9609208'],
 ['4742300', '4789002', '4744004', '4744003'],
 ['4789099', '8640207', '9609208'],
 ['4623109', '4644302', '4683400', '4692300', '4744001', '4744099'],
 ['nan'],
 ['4330499',
  '4611700',
  '4683400',
  '4712100',
  '4724500',
  '4729601',
  '4744001',
  '4744099',
  '4759899',
  '476

In [ ]:
CNAE_secundario_1 = []
CNAE_secundario_2 = []
CNAE_secundario_3 = []
CNAE_secundario_4 = []
CNAE_secundario_5 = []

for indice1 in range(len(lista_cnae)):
    CNAE_secundario_1.append(lista_cnae[indice1][0])
    
    if len(lista_cnae[indice1]) >= 2:
      CNAE_secundario_2.append(lista_cnae[indice1][1])
    else:
      CNAE_secundario_2.append('nan')

    if len(lista_cnae[indice1]) >= 3:
      CNAE_secundario_3.append(lista_cnae[indice1][2])
    else:
      CNAE_secundario_3.append('nan')

    if len(lista_cnae[indice1]) >= 4:
      CNAE_secundario_4.append(lista_cnae[indice1][3])
    else:
      CNAE_secundario_4.append('nan')

    if len(lista_cnae[indice1]) >= 5:
      CNAE_secundario_5.append(lista_cnae[indice1][4])
    else:
      CNAE_secundario_5.append('nan')

df_merge.insert(29,"cnae_sec_1", CNAE_secundario_1, True)
df_merge.insert(30,"cnae_sec_2", CNAE_secundario_2, True)
df_merge.insert(31,"cnae_sec_3", CNAE_secundario_3, True)
df_merge.insert(32,"cnae_sec_4", CNAE_secundario_4, True)
df_merge.insert(33,"cnae_sec_5", CNAE_secundario_5, True)

df_merge.head()

### Separando o CNAE secundário por partes

In [ ]:
cnaes_col = cnaes.loc[:,['Seção',	'Divisão',	'Grupo',	'Classe',	'Subclasse']]
cnaes_col.head()

In [ ]:
cnaes_col['Grupo'] = cnaes_col['Grupo'].str.replace('.','')
cnaes_col['Classe'] = cnaes_col['Classe'].str.replace('.','')
cnaes_col['Classe'] = cnaes_col['Classe'].str.replace('-','')
cnaes_col['Subclasse'] = cnaes_col['Subclasse'].str.replace('-','')
cnaes_col['Subclasse'] = cnaes_col['Subclasse'].str.replace('/','')

In [ ]:
cnaes_col.head()

#### CNAE Secundário 1

In [ ]:
cnaes_col.rename(columns={'Seção': 'Seção1'}, inplace=True)
cnaes_col.rename(columns={'Divisão': 'Divisão1'}, inplace=True)
cnaes_col.rename(columns={'Grupo': 'Grupo1'}, inplace=True)
cnaes_col.rename(columns={'Classe': 'Classe1'}, inplace=True)
cnaes_col.rename(columns={'Subclasse': 'cnae_sec_1'}, inplace=True)

cnaes_col.head()

In [ ]:
df_merge1 = pd.merge(df_merge, cnaes_col, how = 'left', on = 'cnae_sec_1')
df_merge1.head()

#### CNAE Secundário 2

In [ ]:
cnaes_col.rename(columns={'Seção1': 'Seção2'}, inplace=True)
cnaes_col.rename(columns={'Divisão1': 'Divisão2'}, inplace=True)
cnaes_col.rename(columns={'Grupo1': 'Grupo2'}, inplace=True)
cnaes_col.rename(columns={'Classe1': 'Classe2'}, inplace=True)
cnaes_col.rename(columns={'cnae_sec_1': 'cnae_sec_2'}, inplace=True)

cnaes_col.head()

In [ ]:
df_merge1 = pd.merge(df_merge1, cnaes_col, how = 'left', on = 'cnae_sec_2')
df_merge1.head()

#### CNAE Secundário 3

In [ ]:
cnaes_col.rename(columns={'Seção2': 'Seção3'}, inplace=True)
cnaes_col.rename(columns={'Divisão2': 'Divisão3'}, inplace=True)
cnaes_col.rename(columns={'Grupo2': 'Grupo3'}, inplace=True)
cnaes_col.rename(columns={'Classe2': 'Classe3'}, inplace=True)
cnaes_col.rename(columns={'cnae_sec_2': 'cnae_sec_3'}, inplace=True)

cnaes_col.head()

In [ ]:
df_merge1 = pd.merge(df_merge1, cnaes_col, how = 'left', on = 'cnae_sec_3')
df_merge1.head()

#### CNAE Secundário 4

In [ ]:
df_merge1=pd.read_csv('/content/drive/MyDrive/Projetos_ion/ION_Prospeccao_Inteligente/datasets/CNPJ_metadados/df_merge1.csv', dtype=str)
df_merge1.head()

In [ ]:
cnaes_col.rename(columns={'Seção3': 'Seção4'}, inplace=True)
cnaes_col.rename(columns={'Divisão3': 'Divisão4'}, inplace=True)
cnaes_col.rename(columns={'Grupo3': 'Grupo4'}, inplace=True)
cnaes_col.rename(columns={'Classe3': 'Classe4'}, inplace=True)
cnaes_col.rename(columns={'cnae_sec_3': 'cnae_sec_4'}, inplace=True)

cnaes_col.head()

In [ ]:
df_merge1 = pd.merge(df_merge1, cnaes_col, how = 'left', on = 'cnae_sec_4')
df_merge1.head()

#### CNAE Secundário 5

In [ ]:
cnaes_col.rename(columns={'Seção4': 'Seção5'}, inplace=True)
cnaes_col.rename(columns={'Divisão4': 'Divisão5'}, inplace=True)
cnaes_col.rename(columns={'Grupo4': 'Grupo5'}, inplace=True)
cnaes_col.rename(columns={'Classe4': 'Classe5'}, inplace=True)
cnaes_col.rename(columns={'cnae_sec_4': 'cnae_sec_5'}, inplace=True)

cnaes_col.head()

In [ ]:
df_merge1 = pd.merge(df_merge1, cnaes_col, how = 'left', on = 'cnae_sec_5')
df_merge1.head()

In [ ]:
# Cuidado ao gerar essa célula
# df_merge1.to_csv('/content/drive/MyDrive/Projetos_ion/ION_Prospeccao_Inteligente/datasets/Recife Doces/dataset_pe_recifedoces.csv', index=False)

## Etapa 5:


In [ ]:
dataset_path = "/content/drive/MyDrive/Projetos_ion/ION_Prospeccao_Inteligente/datasets/Recife_Doces/dataset_pe_recifedoces.csv"
clients_path = "/content/drive/MyDrive/Projetos_ion/ION_Prospeccao_Inteligente/datasets/Recife_Doces/estabelecimentos_empresas_recifedoces_filial1_todos.csv"

In [ ]:
desired_cols = [
    'cnpj',
    'idade_empresa',
    'capital_social',
    'porte',
    'cnae'
]
cnae_categories = ['Seção', 'Divisão', 'Grupo', 'Classe']
sec_cnaes = [f"cnae_sec_{i + 1}" for i in range(5)]
sec_cnaes_categories = [f"{cat}{i + 1}" for i in range(5) for cat in cnae_categories]

desired_cols.extend(cnae_categories)
desired_cols.extend(sec_cnaes)
desired_cols.extend(sec_cnaes_categories)
desired_cols

In [ ]:
dataset = pd.read_csv(dataset_path, usecols=desired_cols)
dataset.head()

In [ ]:
clients = pd.read_csv(clients_path, dtype=str)
clients.head()

In [ ]:
positive_cnpjs = set(clients['cnpj'].values)
targets = dataset['cnpj'].apply(lambda cnpj: 1 if cnpj in positive_cnpjs else 0)
dataset['target'] = targets

In [ ]:
dataset.head()

In [ ]:
dataset[dataset['target'] == 1].shape

In [ ]:
dataset = dataset.convert_dtypes()
dataset.head()

In [ ]:
dataset_positive = dataset[dataset['target'] == 1]
dataset_positive.head()

In [ ]:
categories = ['Seção', 'Divisão', 'Grupo', 'Classe', 'cnae']

In [ ]:
clients_categories = {}

for cat in categories:
    cat_cols = [col for col in dataset_positive.columns if col.startswith(cat)]
    unique_values = pd.unique(dataset_positive[cat_cols].values.ravel('K'))
    unique_values = sorted([v for v in unique_values if not pd.isna(v)])
    clients_categories[cat] = unique_values  

In [ ]:
clients_categories

In [ ]:
# Pegar os não-clientes
# Para cada categoria
# Pega aqueles que não têm uma das categorias do cliente (target) (target_Seção) (target_Divisão)
# Considera eles como 0


In [ ]:
for cat, values in clients_categories.items():
    cat_cols = [col for col in dataset.columns if col.startswith(cat)]
    cat_targets = pd.np.where(dataset[cat_cols].isin(values).any(1), -1, 0)
    dataset[f'target_{cat}'] = cat_targets
    dataset.loc[dataset['cnpj'].isin(positive_cnpjs), f'target_{cat}'] = 1
    
dataset.head()

In [ ]:
# Cuidado ao rodar essa célula
# dataset.to_csv('/content/drive/MyDrive/Projetos_ion/ION_Prospeccao_Inteligente/datasets/Recife_Doces/recifedoces_filial_target.csv', index=False)

## Etapa 6:

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Projetos_ion/ION_Prospeccao_Inteligente/datasets/Recife_Doces/recifedoces_filial_target.csv')
df.head()

In [ ]:
df = df.convert_dtypes()

### Dados nulos

In [ ]:
df.dropna(subset=['Seção','Grupo'], inplace=True)

In [ ]:
df = df[df['cnae'] != 9900800]
df = df[df['cnae'] != 8888888]

In [ ]:
df['cnae_sec_1'] = df['cnae_sec_1'].fillna(df['cnae'])
df['cnae_sec_2'] = df['cnae_sec_2'].fillna(df['cnae'])
df['cnae_sec_3'] = df['cnae_sec_3'].fillna(df['cnae'])
df['cnae_sec_4'] = df['cnae_sec_4'].fillna(df['cnae'])
df['cnae_sec_5'] = df['cnae_sec_5'].fillna(df['cnae'])

In [ ]:
df['Seção1'] = df['Seção1'].fillna(df['Seção'])
df['Seção2'] = df['Seção2'].fillna(df['Seção'])
df['Seção3'] = df['Seção3'].fillna(df['Seção'])
df['Seção4'] = df['Seção4'].fillna(df['Seção'])
df['Seção5'] = df['Seção5'].fillna(df['Seção'])

In [ ]:
df['Divisão1'] = df['Divisão1'].fillna(df['Divisão'])
df['Divisão2'] = df['Divisão2'].fillna(df['Divisão'])
df['Divisão3'] = df['Divisão3'].fillna(df['Divisão'])
df['Divisão4'] = df['Divisão4'].fillna(df['Divisão'])
df['Divisão5'] = df['Divisão5'].fillna(df['Divisão'])

In [ ]:
df['Grupo1'] = df['Grupo1'].fillna(df['Grupo'])
df['Grupo2'] = df['Grupo2'].fillna(df['Grupo'])
df['Grupo3'] = df['Grupo3'].fillna(df['Grupo'])
df['Grupo4'] = df['Grupo4'].fillna(df['Grupo'])
df['Grupo5'] = df['Grupo5'].fillna(df['Grupo'])

In [ ]:
df['Classe1'] = df['Classe1'].fillna(df['Classe'])
df['Classe2'] = df['Classe2'].fillna(df['Classe'])
df['Classe3'] = df['Classe3'].fillna(df['Classe'])
df['Classe4'] = df['Classe4'].fillna(df['Classe'])
df['Classe5'] = df['Classe5'].fillna(df['Classe'])

In [ ]:
df = df.dropna()

In [ ]:
df['Seção'] = [ord(x) - 64 for x in df['Seção']]
df['Seção1'] = [ord(x) - 64 for x in df['Seção1']]
df['Seção2'] = [ord(x) - 64 for x in df['Seção2']]
df['Seção3'] = [ord(x) - 64 for x in df['Seção3']]
df['Seção4'] = [ord(x) - 64 for x in df['Seção4']]
df['Seção5'] = [ord(x) - 64 for x in df['Seção5']]

In [ ]:
# Cuidado ao rodar esta célula
# df.to_csv('/content/drive/MyDrive/Projetos_ion/ION_Prospeccao_Inteligente/datasets/Recife_Doces/dataset_limpo_recifedoces_filial.csv', index=False)